# Fake News Detection - Model Training Pipeline

This notebook demonstrates the complete training pipeline for the fake news detection model.


In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Add src directory to path
sys.path.append('../src')

# Import our custom classes
from preprocessor import Preprocessor
from feature_extractor import FeatureExtractor
from model_trainer import ModelTrainer


## 1. Load Dataset


In [ ]:
# Load the dataset
# Note: Replace 'your_dataset.csv' with your actual dataset filename
dataset_path = '../data/raw/your_dataset.csv'

try:
    df = pd.read_csv(dataset_path)
    print(f"Dataset loaded successfully!")
    print(f"Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())
except FileNotFoundError:
    print(f"Dataset not found at {dataset_path}")
    print("Please download a dataset and place it in the data/raw/ directory")
    print("Expected columns: 'text' and 'label'")
    print("\nExample dataset structure:")
    print("text,label")
    print("This is a fake news article,1")
    print("This is a real news article,0")


## 2. Data Preprocessing


In [ ]:
# Initialize preprocessor
preprocessor = Preprocessor()

# Apply preprocessing to the text column
print("Preprocessing text data...")
df['cleaned_text'] = df['text'].apply(preprocessor.clean_text)

print("Preprocessing completed!")
print(f"\nOriginal text example:")
print(df['text'].iloc[0])
print(f"\nCleaned text example:")
print(df['cleaned_text'].iloc[0])


## 3. Train-Test Split


In [ ]:
# Split the data into training and testing sets
X = df['cleaned_text'].values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"Training labels distribution: {np.bincount(y_train)}")
print(f"Test labels distribution: {np.bincount(y_test)}")


## 4. Feature Extraction


In [ ]:
# Initialize feature extractor
feature_extractor = FeatureExtractor(method='tfidf')

# Fit and transform training data
print("Generating features for training data...")
X_train_features = feature_extractor.generate_features(X_train, fit=True)

# Transform test data (don't fit on test data)
print("Generating features for test data...")
X_test_features = feature_extractor.generate_features(X_test, fit=False)

print(f"Training features shape: {X_train_features.shape}")
print(f"Test features shape: {X_test_features.shape}")

# Save the fitted vectorizer
vectorizer_path = '../models/tfidf_vectorizer.pkl'
os.makedirs('../models', exist_ok=True)
joblib.dump(feature_extractor.vectorizer, vectorizer_path)
print(f"Vectorizer saved to {vectorizer_path}")


## 5. Model Training


In [ ]:
# Initialize model trainer with Bi-LSTM architecture
# Note: For TF-IDF features, we'll use a simpler architecture since TF-IDF gives us dense features
# But we can still use the ModelTrainer class for consistency
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, LSTM, Embedding
from tensorflow.keras.optimizers import Adam

# Create a model optimized for TF-IDF features with Bi-LSTM-inspired architecture
# We'll use dense layers but with bidirectional-like processing
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_features.shape[1],)),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("Model architecture (optimized for TF-IDF features):")
model.summary()


In [ ]:
# Train the model
print("Training the model...")
history = model.fit(
    X_train_features, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Save the trained model
model_path = '../models/fake_news_model.h5'
model.save(model_path)
print(f"Model saved to {model_path}")


## 6. Model Evaluation


In [ ]:
# Make predictions on test set
y_pred_proba = model.predict(X_test_features)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Real News', 'Fake News']))

# Print confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Real News', 'Fake News'],
            yticklabels=['Real News', 'Fake News'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()


## Alternative: Using Bi-LSTM with Sequence Features

If you want to use the Bi-LSTM architecture with sequence-based features instead of TF-IDF, you can use the ModelTrainer class directly. Here's how:


In [ ]:
# Alternative approach: Using Bi-LSTM ModelTrainer for sequence-based features
# This would require converting text to sequences instead of TF-IDF

# Example of how to use the Bi-LSTM ModelTrainer:
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Tokenize and convert to sequences
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train)
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

# # Pad sequences
# max_length = 100
# X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
# X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# # Use the Bi-LSTM ModelTrainer
# bi_lstm_trainer = ModelTrainer(vocab_size=5000, embedding_dim=128, max_length=max_length)
# history = bi_lstm_trainer.train(X_train_padded, y_train, epochs=20, batch_size=32)

print("Note: The Bi-LSTM ModelTrainer is available in src/model_trainer.py")
print("It uses Bidirectional(LSTM()) for better text understanding!")
